<a href="https://colab.research.google.com/github/Priyanshu27083/dataset_regression/blob/main/pima_diabetes_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("/content/diabetes.csv")
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Outcome',axis = 1), data['Outcome'], test_size = 0.2, random_state = 0)
for col in X_train.columns:
    median = data[col].median()
    data[col] = data[col].replace(0, median)